In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# install requirements

In [ ]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install lxml

## Download user dataset and turn it into dastaset

In [ ]:
import requests
url = "https://archive.org/download/stackexchange/stackoverflow.com-Users.7z"
file_name = "stackoverflow.com-Users.7z"

# Download the file
response = requests.get(url)
with open(file_name, 'wb') as file:
    file.write(response.content)

# Verify that the file has been downloaded
!ls -lh {file_name}
import os
import py7zr
zip_file_path = '/content/drive/MyDrive/stackoverflow.com-Users.7z'

extraction_directory = '/content/drive/MyDrive/Users'
os.makedirs(extraction_directory, exist_ok=True)
with py7zr.SevenZipFile(zip_file_path, mode='r') as z:
    z.extractall(extraction_directory)
# Assuming your XML file is named "example.xml"
xml_file_path = "/content/drive/MyDrive/Users/Users.xml"

# Read the first 10 lines of the XML file
with open(xml_file_path, 'r') as file:
    for i in range(10):
        line = file.readline()
        print(line.strip())


In [ ]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset, concatenate_datasets
from lxml import etree

def convert_xml_to_datasets(xml_file, chunk_size=1000):
    context = etree.iterparse(xml_file, events=("start", "end"))
    _, root = next(context)

    total_rows = 0
    datasets_chunks = []

    rows_chunk = []
    for event, elem in context:
        if event == "start" and elem.tag == "row":
            row = {
                "Id": elem.get("Id"),
                "Reputation": elem.get("Reputation"),
                "CreationDate": elem.get("CreationDate"),
                "DisplayName": elem.get("DisplayName"),
                "Mobile Development": elem.get("Mobile Development"),
                "LastAccessDate": elem.get("LastAccessDate"),
                "AboutMe": elem.get("AboutMe"),
                "Views": elem.get("Views"),
                "UpVotes": elem.get("UpVotes"),
                "DownVotes": elem.get("DownVotes"),
            }
            rows_chunk.append(row)
            total_rows += 1

        if total_rows > 0 and total_rows % chunk_size == 0:
            # Process the chunk
            datasets_chunks.append(Dataset.from_pandas(pd.DataFrame(rows_chunk)))

            # Reset the context for the next chunk
            root.clear()
            while root.getprevious() is not None:
                del root.getparent()[0]

            rows_chunk = []

    # Process the remaining rows (not forming a complete chunk)
    if total_rows % chunk_size != 0:
        datasets_chunks.append(Dataset.from_pandas(pd.DataFrame(rows_chunk)))

    # Concatenate the processed chunks into a single dataset
    final_dataset = concatenate_datasets(datasets_chunks)


    return final_dataset

# Replace 'your_xml_file_path' with the path to your XML file
xml_file_path = '/content/drive/MyDrive/Users/Users.xml'
final_dataset = convert_xml_to_datasets(xml_file_path, chunk_size=1000)


## save dataset in local

In [ ]:
from datasets import load_dataset
import shutil
def zip_datas(dataset,drive_path, zip_file_path):
  comment_dataset.save_to_disk(drive_path)
  # Create a ZIP archive of the dataset
  shutil.make_archive(zip_file_path[:-4], 'zip', source_dir)

## export username and comments data from local

In [ ]:
import zipfile
from datasets import load_dataset
def export_data_set(zip_file_path, extracted_folder_path):
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

def load_data(directory_path):
  return load_dataset('arrow', data_files=directory_path)

In [ ]:
#comment_dataset
zip_file_path = '/content/drive/MyDrive/filtered_comment_dataset.zip'
extracted_folder_path = '/content/extracted_comment_dataset'
export_data_set(zip_file_path, extracted_folder_path)
directory_path = '/content/extracted_comment_dataset/data-00000-of-00001.arrow'
loaded_dataset = load_data(directory_path)

In [ ]:
#user_dataset
zip_file_path = '/content/drive/MyDrive/filtered_dataset.zip'
extracted_folder_path = '/content/extracted_dataset'
export_data_set(zip_file_path, extracted_folder_path)
directory_path = '/content/extracted_dataset/data-00000-of-00001.arrow'
final_dataset = load_data(directory_path)

In [ ]:
#filter user data set in order to keep just the users that are in comments section(UserId)
train_ids_ste = set(loaded_dataset['train']['UserId'])
filtered_dataset = final_dataset.filter(lambda example : example['Id']) in train_ids_ste)

Filter:   0%|          | 0/337811 [00:00<?, ? examples/s]

In [ ]:
print(filtered_dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'Mobile Development', 'LastAccessDate', 'AboutMe', 'Views', 'UpVotes', 'DownVotes'],
        num_rows: 337811
    })
})


In [ ]:
#to find mentions
def is_reply(comment):
    return '@' in comment
train_dataset = loaded_dataset['train']
# Apply the function to the 'Text' column to identify reply comments
train_dataset = train_dataset.map(lambda example: {'IsReply': is_reply(example['Text'])})

Map:   0%|          | 0/2017836 [00:00<?, ? examples/s]

ALL REPLIES DATA WITH @

In [ ]:
isreplied_test = train_dataset.filter(lambda example: example['IsReply'])

Filter:   0%|          | 0/2017836 [00:00<?, ? examples/s]

In [ ]:
#Add username in order to eliminate spaces
trained_dataset = filtered_dataset


# Apply the function to the 'Text' column to identify reply comments
filtered_final_dataset= trained_dataset.map(lambda example: {'Username':  "".join(example['DisplayName'].split())})


Map:   0%|          | 0/337811 [00:00<?, ? examples/s]

## extract all mentions and make a unique list out of it

In [ ]:
import re
from concurrent.futures import ThreadPoolExecutor

# Load your dataset
your_dataset = isreplied_test

# Function to extract mentions from a single text
def extract_mentions(text):
    return re.findall(r'@(\w+)', text)

# Specify the text column in your dataset
text_column = 'Text'  # Replace 'your_text_column' with the actual column containing the text

# Extract mentions from each text in parallel
all_mentions = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(extract_mentions, example[text_column]) for example in your_dataset]

    for future in futures:
        mentions = future.result()
        all_mentions.extend(mentions)

# print("All Mentions:", all_mentions)


In [ ]:
myset = set(all_mentions)

In [ ]:
print(len(myset))
mylist = list(myset)

105848


create a dictionary for each username to id

In [ ]:
from collections import defaultdict
comment_dataset = train_dataset
post_comments_dict = defaultdict(set)

# Function to add comments to the dictionary
def add_comment_to_dict(example, post_comments_dict):
    post_comments_dict[example['PostId']].add(example['UserId'])

# Iterate through the examples in the dataset
for example in comment_dataset:
    add_comment_to_dict(example, post_comments_dict)

# Convert the defaultdict to a regular dictionary
post_comments_dict = dict(post_comments_dict)

# # Display the dictionary
# for post_id, comments in post_comments_dict.items():
#     print(f"PostId: {post_id}, Comments: {comments}")



In [ ]:
for post_id, comments in list(post_comments_dict.items())[:5]:
    print(f"PostId: {post_id}, Comments: {comments}")

PostId: 51979, Comments: {'379'}
PostId: 8637, Comments: {'27067', '119046', '257493', '4504', '1386054', '4906', '29493', '21574', '10080', '383', '42048', '1010847'}
PostId: 52332, Comments: {'31325', '5321'}
PostId: 52383, Comments: {'5182'}
PostId: 8489, Comments: {'4906', '191507', '10080', '428876', '9330', '80363'}


In [ ]:
# Function to extract mentions from a single text
def extract_mentions(text):
    return re.findall(r'@(\w+)', text)

In [ ]:
print(train_dataset)

Dataset({
    features: ['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId', 'ContentLicense', 'IsReply'],
    num_rows: 2017836
})


In [ ]:
print(filtered_final_dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'Mobile Development', 'LastAccessDate', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'Username'],
        num_rows: 337811
    })
})


In [ ]:
comment_dataset = train_dataset
user_dataset=filtered_final_dataset
usernamedict = {}
import pandas as pd

# Convert your datasets to pandas DataFrames (assuming they are not too large)
comment_df = comment_dataset.to_pandas()
user_df = user_dataset['train'].to_pandas()

# Create a set of unique usernames from your list
usernames_set = set(mylist)

# Filter the user DataFrame for all matching usernames
filtered_users = user_df[user_df['Username'].isin(usernames_set)]

# Create a dictionary mapping usernames to lists of corresponding IDs
usernamedict = filtered_users.groupby('Username')['Id'].apply(list).to_dict()

# Display the result
print(usernamedict)



{'0009laH': ['6538278'], '001': ['103264', '669576'], '00jt': ['1030479'], '0100100001101001': ['2955004'], '010011100101': ['10914284'], '0101': ['1244005'], '0101100101': ['2964379'], '01jayss': ['622481'], '03Usr': ['484105'], '04FS': ['10955263'], '08442': ['1414515'], '0909EM': ['843387'], '0__': ['515054'], '0___________': ['6110094'], '0aslam0': ['1929187'], '0b10011': ['526741'], '0bj3ct': ['2090332'], '0gravity': ['1478722'], '0lan': ['12613946'], '0m3r': ['4539709'], '0p3r4t0r': ['8353926'], '0x141E': ['1318136'], '0x24a537r9': ['275792'], '0x263A': ['16450169'], '0x5453': ['3282436'], '0x5929': ['7347545'], '0x5a4d': ['3587592'], '0x6B6F77616C74': ['1349442'], '0x8badf00d': ['542425'], '0x90': ['142137', '1031417'], '0xC0000022L': ['476371'], '0xC0DEFACE': ['126163'], '0xCAFEBABE': ['494501'], '0xDEADC0DE': ['3275754'], '0xF4D3C0D3': ['7899226'], '0xFADE': ['901389'], '0xSina': ['903643'], '0xTim': ['1823705'], '0xbe5077ed': ['1911388'], '0xced': ['21698'], '0yeoj': ['490356

In [ ]:
def get_reply_info(comment, user_dataset):
    if comment['IsReply']:
        usernames = extract_mentions(comment['Text'])

        matching_users = []
        if usernames and usernames[0] in usernamedict:
            matching_users.extend(usernamedict[usernames[0]])

        userswhopostcomment = post_comments_dict[comment["PostId"]]
        common_users = list(set(userswhopostcomment) & set(matching_users))

        if common_users:
            return common_users
        else:
            return None
    return None

# Assuming train_dataset is a dataset with a 'train' split
comment_dataset = train_dataset.map(
    lambda example: {'repliedID': get_reply_info(example, filtered_final_dataset)}
)

# Display a few examples
comment_dataset[:5]


Map:   0%|          | 0/2017836 [00:00<?, ? examples/s]

{'Id': ['899', '1014', '1022', '1036', '1510'],
 'PostId': ['51979', '8637', '52332', '52383', '8489'],
 'Score': ['0', '45', '1', '1', '4'],
 'Text': ["this looks like what i'm looking for. I'll try it out and report back :)",
  "VERSION THAT WORKS WITHOUT JAVASCRIPT\nHow about if you did this with ASP, etc. and had a timestamp for when the form page was loaded and then compared that to the time when the form was submitted.  If ElapsedTime<10 sec then it's likely spam.",
  "As long as your xml schema is set to the schema file, it should validate it automatically and any error will be detected.  You'll get the benefit of XML intellisense as well if you have a schema associated with the xml.",
  'I should have clarified I wanted to use the IDE and not code.',
  "Explanation of Honeypot Captcha (which looks very good):\nBots love forms. They fill out all the fields. A honeypot Captcha includes a field that is HIDDEN by CSS so only the bots (and those with IE 3.0) see it. If it's filled, 

In [ ]:
myresult = comment_dataset.filter(lambda x : x['repliedID'] is not None  )

Filter:   0%|          | 0/2017836 [00:00<?, ? examples/s]

In [ ]:
print(myresult[0:5])

{'Id': ['140878', '146828', '149273', '158399', '173760'], 'PostId': ['289410', '298452', '298452', '282857', '104896'], 'Score': ['10', '0', '0', '0', '0'], 'Text': ['@Otherside, You have to let go of the idea that interfaces are contracts.  The sad truth is they are not behavior contracts.  They are simply data passing contracts.  Duck typing is no worse in this respect that explicitly implemented interfaces.', "@dtroy - i've added code that will invalidate when another control on in the parents.controls collection paints.. it won't work as well for textbox or comboboxs but you can see what i've done to cover them. not sure if that will do for your needs but you could make it work for what your discribing.", "@dtroy - added the detach for parentHooked. Just noticed this has a few problems if you do a diagonal.. say if you want to select a control thats been covered by the line you can't because the line control is covering it.. i suppose you could relay the mouse messages somehow but

In [ ]:
x = comment_dataset.filter(lambda x : x['repliedID'] is not None and len(x['repliedID'])>=2 )

Filter:   0%|          | 0/2017836 [00:00<?, ? examples/s]

## save new comment_dataset

In [ ]:
from google.colab import drive
from datasets import load_dataset

# Specify the path where you want to save the dataset
drive_path = '/content/drive/MyDrive/c2'

# Save the dataset to Google Drive
comment_dataset.save_to_disk(drive_path)

Saving the dataset (0/1 shards):   0%|          | 0/2017836 [00:00<?, ? examples/s]

In [ ]:
import shutil


source_dir = '/content/drive/MyDrive/c2'

# Destination ZIP file
zip_file_path = '/content/filtered_comment_dataset_2.zip'

# Create a ZIP archive of the dataset
shutil.make_archive(zip_file_path[:-4], 'zip', source_dir)

'/content/filtered_comment_dataset_2.zip'

In [ ]:
from google.colab import files
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>